In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
from scipy import signal

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GroupKFold, GridSearchCV  , StratifiedGroupKFold, cross_validate
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor, MLPClassifier

from random import sample, shuffle
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone
from copy import deepcopy

import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
dataset_path_root = "/home/bruno/Academico/Doctorado/Neuro_Fairness/Shu_Dataset/004_Dataset_6_Random_Feature_Extraction/"
participants=["sub-001","sub-002","sub-003","sub-004","sub-005",
              "sub-006","sub-007","sub-008","sub-009","sub-010",
              "sub-011","sub-012","sub-013","sub-014","sub-015",
              "sub-016","sub-017","sub-018","sub-019","sub-020",
              "sub-021","sub-022","sub-023","sub-024","sub-025"]
sessions = ["ses-01","ses-02","ses-03","ses-04","ses-05"]

In [5]:
dataset={}
for participant in participants:
    dataset[participant]={}
    data_path=participant+"_task_motorimagery_eeg_preprocessing_random_extractor_feature_mean.mat"
    data=sio.loadmat(dataset_path_root + data_path)
    for session in sessions:
        dataset[participant][session +'_data_random']=data[session +'_data_random_mean']
        dataset[participant][session +'_labels_trials']=data[session +'_labels_trials']
    dataset[participant]['sfreq']=np.squeeze(data['sfreq'])
    dataset[participant]['age']=np.squeeze(data['age'])
    dataset[participant]['gender']=data['gender'][0]
    dataset[participant]['group_medidator']=data['group_medidator'][0]
    dataset[participant]['id_participant']=data['id_participant'][0]

In [6]:
index_female = []
index_male = []
for participant in participants:
    if dataset[participant]['gender'] == 'M':
        index_male.append(participant)
    elif dataset[participant]['gender'] == 'F':
        index_female.append(participant)
print(f"Participantes hombres: {index_male}")
print(f"Participantes mujeres: {index_female}")

Participantes hombres: ['sub-001', 'sub-002', 'sub-008', 'sub-012', 'sub-013', 'sub-015', 'sub-017', 'sub-018', 'sub-019', 'sub-021', 'sub-022', 'sub-023', 'sub-025']
Participantes mujeres: ['sub-003', 'sub-004', 'sub-005', 'sub-006', 'sub-007', 'sub-009', 'sub-010', 'sub-011', 'sub-014', 'sub-016', 'sub-020', 'sub-024']


In [9]:
for participant in participants:
    data_ = np.concatenate((dataset[participant]['ses-01_data_random'],
                            dataset[participant]['ses-02_data_random'],
                            dataset[participant]['ses-03_data_random'],
                            dataset[participant]['ses-04_data_random'],
                            dataset[participant]['ses-05_data_random']),axis=0)
    
    dataset[participant]['data_random'] = data_
    dataset[participant]['data_gender'] =  np.array(list(dataset[participant]['gender']) * data_.shape[0])
    dataset[participant]['group_participant'] =  np.array(list([participant]) * data_.shape[0])

In [10]:
n_features = data_.shape[-1]
N_it = 20
n_test_participant = 2
n_val_participant = 2 
n_ign_participant = 1

max_iter = 200
patience = 50 
info_exp = {}
info_clfs = {}
for it in range(N_it):
    dic_aux = {}
    
    X_train = None
    X_val = None
    X_test = None 
    
    X_train_ = None
    X_val_ = None
    X_test_ = None
    
    idx_male = index_male.copy()
    idx_female = index_female.copy()
    
    # TEST PARTICIPANTS:
    idx_male_test = sample(idx_male, n_test_participant)
    idx_female_test = sample(idx_female, n_test_participant)
    for it_ in range(n_test_participant):
        idx_male.remove(idx_male_test[it_])
        idx_female.remove(idx_female_test[it_])
    idx_test = idx_male_test + idx_female_test
    dic_aux['reg_idx_test'] = idx_test   
    
    # VALIDATION PARTICIPANTS:
    idx_male_val = sample(idx_male, n_val_participant)
    idx_female_val = sample(idx_female, n_val_participant)
    for it_ in range(n_val_participant):
        idx_male.remove(idx_male_val[it_])
        idx_female.remove(idx_female_val[it_])
    idx_val = idx_male_val + idx_female_val
    dic_aux['reg_idx_val'] = idx_val  
    
    # TRAIN PARTICIPANTS:
    idx_male_ignore = sample(idx_male, n_ign_participant)
    for it_ in range(n_ign_participant):
        idx_male.remove(idx_male_ignore[it_])
    idx_male_train = idx_male.copy()
    idx_female_train = idx_female.copy()
    idx_train = idx_male_train + idx_female_train
    dic_aux['reg_idx_train'] = idx_train    
        
    
    # CONCATENAMOS EL CONJUNTO DE DATOS
    # TEST
    X_test = np.zeros((1,n_features))
    Y_test = np.zeros(1)
    for participant in idx_test:
        X_test = np.concatenate((X_test, dataset[participant]['data_random']),axis=0)
        Y_test = np.concatenate((Y_test, dataset[participant]['data_gender']),axis=0)
    X_test = X_test[1:,:]
    Y_test = Y_test[1:]
    dic_aux['n_trials_test'] = {'male':np.sum(Y_test == 'M'),'female':np.sum(Y_test == 'F')}
    dic_aux['proportion_trials_test'] = {'male':np.sum(Y_test == 'M')/(np.sum(Y_test == 'M')+np.sum(Y_test == 'F')),
                                      'female':np.sum(Y_test == 'F')/(np.sum(Y_test == 'M')+np.sum(Y_test == 'F'))}
    # VALIDATION
    X_val = np.zeros((1,n_features))
    Y_val = np.zeros(1)
    for participant in idx_val:
        X_val = np.concatenate((X_val, dataset[participant]['data_random']),axis=0)
        Y_val = np.concatenate((Y_val, dataset[participant]['data_gender']),axis=0)
    X_val = X_val[1:,:]
    Y_val = Y_val[1:]
    dic_aux['n_trials_val'] = {'male':np.sum(Y_val == 'M'),'female':np.sum(Y_val == 'F')}
    dic_aux['proportion_trials_val'] = {'male':np.sum(Y_val == 'M')/(np.sum(Y_val == 'M')+np.sum(Y_val == 'F')),
                                      'female':np.sum(Y_val == 'F')/(np.sum(Y_val == 'M')+np.sum(Y_val == 'F'))}
        
    # TRAIN
    X_train = np.zeros((1,n_features))
    Y_train = np.zeros(1)
    for participant in idx_train:
        X_train = np.concatenate((X_train, dataset[participant]['data_random']),axis=0)
        Y_train = np.concatenate((Y_train, dataset[participant]['data_gender']),axis=0)
    X_train = X_train[1:,:]
    Y_train = Y_train[1:]
    dic_aux['n_trials_train'] = {'male':np.sum(Y_train == 'M'),'female':np.sum(Y_train == 'F')}
    dic_aux['proportion_trials_train'] = {'male':np.sum(Y_train == 'M')/(np.sum(Y_train == 'M')+np.sum(Y_train == 'F')),
                                      'female':np.sum(Y_train == 'F')/(np.sum(Y_train == 'M')+np.sum(Y_train == 'F'))}   
    
    info_exp[f'it_{it}']=dic_aux
    dic_aux={}
    
    #-----------------------CLASSIFIER 1 -----------------------------#
    acc_train = None
    acc_val = None
    acc_test = None
    
    tol = 1e-4
    scaler = StandardScaler()
    clf = LinearDiscriminantAnalysis(tol=tol)
    
    X_train_ = scaler.fit_transform(X_train, Y_train)
    X_val_ = scaler.transform(X_val)
    X_test_ = scaler.transform(X_test)
    
    
    clf.fit(X_train_, Y_train)
    acc_train = clf.score(X_train_ , Y_train)
    acc_val = clf.score(X_val_ , Y_val)
    acc_test = clf.score(X_test_, Y_test)
    
    dic_aux['clf_1']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test}
    
    #-----------------------CLASSIFIER 2 -----------------------------#
    acc_train = None
    acc_val = None
    acc_test = None
    
    tol = 1e-4
    scaler = StandardScaler()
    clf = svm.SVC(C=1.0, kernel='linear', tol=tol)
    
    X_train_ = scaler.fit_transform(X_train, Y_train)
    X_val_ = scaler.transform(X_val)
    X_test_ = scaler.transform(X_test)
    
    
    clf.fit(X_train_, Y_train)
    acc_train = clf.score(X_train_ , Y_train)
    acc_val = clf.score(X_val_ , Y_val)
    acc_test = clf.score(X_test_, Y_test)
    
    dic_aux['clf_2']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test}
    
    #-----------------------CLASSIFIER 3 -----------------------------#
    acc_train = None
    acc_val = None
    acc_test = None
    
    tol = 1e-4
    scaler = StandardScaler()
    clf = svm.SVC(C=1.0, kernel='rbf', tol=tol)
    
    X_train_ = scaler.fit_transform(X_train, Y_train)
    X_val_ = scaler.transform(X_val)
    X_test_ = scaler.transform(X_test)
    
    
    clf.fit(X_train_, Y_train)
    acc_train = clf.score(X_train_ , Y_train)
    acc_val = clf.score(X_val_ , Y_val)
    acc_test = clf.score(X_test_, Y_test)
    
    dic_aux['clf_3']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test}
    
    
    #-----------------------CLASSIFIER 4 -----------------------------#
    acc_train_scores = []
    acc_val_scores = []
    acc_train = 0
    acc_val = 0
    acc_test = 0
    
    # Counter for patience
    it_patience = 0
    
    # Best classifier in early stopping
    best_clf = None
    it_stop = 0 
    
    scaler = StandardScaler()
    clf = MLPClassifier(hidden_layer_sizes=(10,8,5), activation='relu', solver='adam', alpha=0.0001,
                        learning_rate='constant', learning_rate_init=0.01,max_iter=max_iter)
    classes = np.unique(Y_train)

    for it_ in range(max_iter):
        # Scaler fit/transform
        scaler.partial_fit(X_train, Y_train)
        X_train_ = scaler.transform(X_train)
        X_val_ = scaler.transform(X_val)
        
        # Classifier fit / evaluate
        clf.partial_fit(X_train_,Y_train,classes=classes)
        acc_train_scores.append(clf.score(X_train_,Y_train))
        acc_val_scores.append(clf.score(X_val_,Y_val))
        
        if acc_val_scores[-1] >= acc_val:
            acc_val = acc_val_scores[-1]
            best_clf = deepcopy(clf)
            it_stop = it_
            it_patience = 0
        else: 
            it_patience += 1
        
        if it_patience >= patience:
            break
        
    X_test_ = scaler.transform(X_test)    
    acc_test = best_clf.score(X_test_,Y_test)
    acc_train = acc_train_scores[it_stop] 
     
    dic_aux['clf_4']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test,
                      'acc_train_scores': acc_train_scores, 'acc_val_scores': acc_val_scores, 'it_stop':it_stop}
    
    #-----------------------CLASSIFIER 5 -----------------------------#
    acc_train_scores = []
    acc_val_scores = []
    acc_train = 0
    acc_val = 0
    acc_test = 0
    
    # Counter for patience
    it_patience = 0
    
    # Best classifier in early stopping
    best_clf = None
    it_stop = 0 
    
    scaler = StandardScaler()
    clf = MLPClassifier(hidden_layer_sizes=(10,8,3), activation='relu', solver='adam', alpha=0.0001,
                        learning_rate='constant', learning_rate_init=0.01,max_iter=max_iter)
    classes = np.unique(Y_train)

    for it_ in range(max_iter):
        # Scaler fit/transform
        scaler.partial_fit(X_train, Y_train)
        X_train_ = scaler.transform(X_train)
        X_val_ = scaler.transform(X_val)
        
        # Classifier fit / evaluate
        clf.partial_fit(X_train_,Y_train,classes=classes)
        acc_train_scores.append(clf.score(X_train_,Y_train))
        acc_val_scores.append(clf.score(X_val_,Y_val))
        
        if acc_val_scores[-1] >= acc_val:
            acc_val = acc_val_scores[-1]
            best_clf = deepcopy(clf)
            it_stop = it_
            it_patience = 0
        else: 
            it_patience += 1
        
        if it_patience >= patience:
            break
        
    X_test_ = scaler.transform(X_test)    
    acc_test = best_clf.score(X_test_,Y_test)
    acc_train = acc_train_scores[it_stop] 
     
    dic_aux['clf_5']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test,
                      'acc_train_scores': acc_train_scores, 'acc_val_scores': acc_val_scores, 'it_stop':it_stop}
    
    #-----------------------CLASSIFIER 6 -----------------------------#
    acc_train_scores = []
    acc_val_scores = []
    acc_train = 0
    acc_val = 0
    acc_test = 0
    
    # Counter for patience
    it_patience = 0
    
    # Best classifier in early stopping
    best_clf = None
    it_stop = 0 
    
    scaler = StandardScaler()
    clf = MLPClassifier(hidden_layer_sizes=(10,4,2), activation='relu', solver='adam', alpha=0.0001,
                        learning_rate='constant', learning_rate_init=0.01,max_iter=max_iter)
    classes = np.unique(Y_train)

    for it_ in range(max_iter):
        # Scaler fit/transform
        scaler.partial_fit(X_train, Y_train)
        X_train_ = scaler.transform(X_train)
        X_val_ = scaler.transform(X_val)
        
        # Classifier fit / evaluate
        clf.partial_fit(X_train_,Y_train,classes=classes)
        acc_train_scores.append(clf.score(X_train_,Y_train))
        acc_val_scores.append(clf.score(X_val_,Y_val))
        
        if acc_val_scores[-1] >= acc_val:
            acc_val = acc_val_scores[-1]
            best_clf = deepcopy(clf)
            it_stop = it_
            it_patience = 0
        else: 
            it_patience += 1
        
        if it_patience >= patience:
            break
        
    X_test_ = scaler.transform(X_test)    
    acc_test = best_clf.score(X_test_,Y_test)
    acc_train = acc_train_scores[it_stop] 
     
    dic_aux['clf_6']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test,
                      'acc_train_scores': acc_train_scores, 'acc_val_scores': acc_val_scores, 'it_stop':it_stop}
    
    info_clfs[f'it_{it}']=dic_aux

In [11]:
#Generamos la tabla de los acc para cada iteración y clasificador
n_clfs = 6

matrix_acc_train = np.zeros((n_clfs,N_it))
matrix_acc_val = np.zeros((n_clfs,N_it))
matrix_acc_test = np.zeros((n_clfs,N_it))

for it0 in range(N_it):
    for it1 in range(n_clfs):
        matrix_acc_train[it1,it0] = info_clfs[f'it_{it0}'][f'clf_{it1+1}']['acc_train']
        matrix_acc_val[it1,it0] = info_clfs[f'it_{it0}'][f'clf_{it1+1}']['acc_val']
        matrix_acc_test[it1,it0] = info_clfs[f'it_{it0}'][f'clf_{it1+1}']['acc_test']

acc_train_clfs_mean = matrix_acc_train.mean(axis=1,keepdims=True)
acc_train_clfs_std = np.std(matrix_acc_train,axis=1,keepdims=True)

acc_val_clfs_mean = matrix_acc_val.mean(axis=1,keepdims=True)
acc_val_clfs_std = np.std(matrix_acc_val,axis=1,keepdims=True)

acc_test_clfs_mean = matrix_acc_test.mean(axis=1,keepdims=True)
acc_test_clfs_std = np.std(matrix_acc_test,axis=1,keepdims=True)

In [12]:
accs_train_df = pd.DataFrame(matrix_acc_train,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6'])
accs_val_df = pd.DataFrame(matrix_acc_val,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6'])
accs_test_df = pd.DataFrame(matrix_acc_test,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6'])
accs_test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
clf_1,0.498947,0.516842,0.509701,0.525146,0.506480,0.505455,0.558792,0.494111,0.504141,0.500541,0.507862,0.534483,0.506037,0.542175,0.503416,0.505679,0.523238,0.508035,0.524701,0.495838
clf_2,0.512632,0.504211,0.523335,0.526734,0.489373,0.501299,0.486229,0.510171,0.493789,0.505405,0.509958,0.535528,0.504987,0.557029,0.485549,0.502975,0.542037,0.511146,0.518981,0.493236
clf_3,0.589474,0.567368,0.553225,0.532557,0.569725,0.588052,0.579449,0.500535,0.511905,0.525946,0.423480,0.434692,0.404724,0.615385,0.478718,0.505679,0.557180,0.497667,0.533541,0.514048
clf_4,0.568421,0.521053,0.518616,0.536792,0.560912,0.497662,0.537606,0.482869,0.481366,0.501622,0.417191,0.494253,0.391601,0.561804,0.458749,0.489995,0.478851,0.463971,0.515861,0.504162
clf_5,0.492632,0.506316,0.539067,0.500794,0.571799,0.457143,0.534428,0.487152,0.503106,0.517297,0.395702,0.502612,0.386877,0.550663,0.467157,0.495403,0.537859,0.510627,0.548102,0.517690
clf_6,0.522105,0.489474,0.511799,0.483325,0.495594,0.497662,0.497881,0.506424,0.482919,0.492432,0.418763,0.469697,0.382677,0.552785,0.471886,0.487831,0.495039,0.497149,0.535101,0.499480


In [13]:
results_df = pd.DataFrame(np.concatenate((acc_train_clfs_mean,acc_train_clfs_std,
                                          acc_val_clfs_mean,acc_val_clfs_std,
                                          acc_test_clfs_mean,acc_test_clfs_std),axis=1),
                          columns = ['TRAIN MEAN', 'TRAIN STD','VAL MEAN', 'VAL STD','TEST MEAN', 'TEST STD']
                          ,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6'])
results_df

,TRAIN MEAN,TRAIN STD,VAL MEAN,VAL STD,TEST MEAN,TEST STD
clf_1,0.514819,0.005753,0.498516,0.013181,0.513581,0.016194
clf_2,0.515265,0.007951,0.498810,0.018656,0.510730,0.018389
clf_3,0.599736,0.014141,0.508296,0.055037,0.524168,0.055719
clf_4,0.549989,0.026887,0.534101,0.041572,0.499168,0.044080
clf_5,0.543626,0.027276,0.530499,0.036705,0.501121,0.045699
clf_6,0.529227,0.032191,0.524682,0.038647,0.489501,0.035710


In [14]:
from scipy.stats import ks_1samp
from scipy.stats import ttest_1samp
from scipy.stats import wilcoxon
from scipy import stats

In [15]:
alpha = 0.05
alpha_bonferroni = alpha / 6

test_norm = pd.DataFrame(columns=['statistics','p-value','H0'])
clf_names = accs_test_df.index.values.tolist()

for clf in clf_names: 
    acc_test = accs_test_df.loc[clf]
    norm_test = ks_1samp(acc_test,stats.norm.cdf)
    if norm_test.pvalue <= alpha_bonferroni:
        test_norm.loc[clf]=[norm_test.statistic,norm_test.pvalue,False]
    else:
        test_norm.loc[clf]=[norm_test.statistic,norm_test.pvalue,True]

print(f'Significancia: {alpha_bonferroni:.2}')
print(f'Intervalo de confianza: {100*(1-alpha_bonferroni)}')
test_norm

Significancia: 0.0083
Intervalo de confianza: 99.16666666666667


,statistics,p-value,H0
clf_1,0.689386,5.355727e-10,False
clf_2,0.686357,6.711657e-10,False
clf_3,0.657160,5.381548e-09,False
clf_4,0.652323,7.480706e-09,False
clf_5,0.650576,8.417214e-09,False
clf_6,0.649020,9.345075e-09,False


In [16]:
mu = 0.5
test = pd.DataFrame(columns=['statics','p-value','H0'])

for clf in clf_names: 
    acc_test = accs_test_df.loc[clf]
    w_test = wilcoxon(acc_test-mu)
    if w_test.pvalue <= alpha_bonferroni:
        test.loc[clf]=[w_test.statistic,w_test.pvalue,False]
    else: 
        test.loc[clf]=[w_test.statistic,w_test.pvalue,True]

print(f'Significancia: {alpha_bonferroni:.2}')
print(f'Intervalo de confianza: {100*(1-alpha_bonferroni)}')
test

Significancia: 0.0083
Intervalo de confianza: 99.16666666666667


,statics,p-value,H0
clf_1,15.0,0.000261,False
clf_2,50.0,0.039989,True
clf_3,54.0,0.058258,True
clf_4,104.0,0.985435,True
clf_5,83.0,0.430433,True
clf_6,66.0,0.153646,True
